# 実行環境の作成

## Google Drive

Google Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Google Cloud Storage

下記コードでGCPに接続

In [2]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

## 環境変数

In [5]:
RESAS_API_KEY = '	02JYzLqUOfTNVfiTWjLlR2g0YwEQPQ7caYxcnZaZ'

# 都道府県一覧

In [6]:
import json
import urllib.request

def get_resas_preflist():
    url = 'https://opendata.resas-portal.go.jp/api/v1/prefectures'
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())
    return res['result']

In [ ]:
preflist = get_resas_preflist()
print(preflist)

GoogleDriveとGoogleCloudStorageに保存

In [8]:
# 保存先パスの設定
path = "resas/preflist.json"

# Google Driveへ保存
drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)

with open(drive_path, 'w') as f:
    json.dump(preflist,f,ensure_ascii=False)

# GoogleCloudStorage
gcs_path = "statistics-hyogo/{}".format(path)

with open(gcs_path, 'w') as f:
    json.dump(preflist,f,ensure_ascii=False)

# 市区町村一覧

In [9]:
import json
import urllib.request

def get_resas_citylist(prefCode=0,designatedCity='all'):
  
    # 都道府県'prefCode'を指定
    # 0の場合は全都道府県の市区町村
    if prefCode == 0:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities'
    else:
      url = 'https://opendata.resas-portal.go.jp/api/v1/cities?prefCode={}'.format(prefCode)
    
    req = urllib.request.Request(url, headers={'X-API-KEY': RESAS_API_KEY})
    with urllib.request.urlopen(req) as response:
        res = json.loads(response.read().decode())


    # 政令指定都市　の指定　'join':政令指定都市統合　'split'：政令指定都市分割 'all'：全て
    if designatedCity == 'join':
      return list(filter(lambda x: x['bigCityFlag'] != '1', res['result']))
    elif designatedCity == 'split':
      return list(filter(lambda x: x['bigCityFlag'] != '2', res['result']))
    else:
      return res['result']

In [ ]:
city1 = get_resas_citylist()
print(city1)

city2 = get_resas_citylist(designatedCity='join')
print(city2)

city3 = get_resas_citylist(prefCode=28,designatedCity='split')
print(city3)

city4 = get_resas_citylist(prefCode=28)
print(city4)

GoogleDriveとGoogleCloudStorageに保存

In [11]:
citylist = get_resas_citylist()

# 保存先パスの設定
path = "resas/citylist.json"

# Google Driveへ保存
drive_path = "drive/MyDrive/statistics-hyogo/{}".format(path)

with open(drive_path, 'w') as f:
    json.dump(citylist,f,ensure_ascii=False)

# GoogleCloudStorage
gcs_path = "statistics-hyogo/{}".format(path)

with open(gcs_path, 'w') as f:
    json.dump(citylist,f,ensure_ascii=False)

# estat-APIで利用したい場合

In [ ]:
import json
import urllib.request

def generate_estat_areas(governmentType='prefecture',prefCode=0):

  # 都道府県の場合
  if governmentType == 'prefecture':
    pref_codes = [d.get('prefCode') for d in get_resas_preflist()]
    return [f'{n:02}'+'000' for n in pref_codes]
  
  # 市区町村の場合
  else:
    city_codes = [d.get('cityCode') for d in get_resas_citylist(prefCode)]
    return city_codes

In [ ]:
areas = generate_estat_areas()
print(areas)

areas2 = generate_estat_areas(governmentType='city')
print(areas2)

areas3 = generate_estat_areas(governmentType='city',prefCode=28)
print(areas3)